# Testing claims made by Judicial Watch using census and registration data.
### A self-described "conservative, non-partisan" government watchdog group alleged 19 counties nationwide are not updating their voter rolls. 
### Judicial Watch threatened the counties, including Bucks, with a lawsuit if they didn't address what the group claimed were "abusrdly high" registration rates and other issues.
### The group claimed their analysis showed 378 counties had a total of 2.5 million "extra" voters.
### Using the same data referenced in the legal letters to four counties in Pennsylvania, this news organization was unable to recreate their results.
### The group has not released a full verison of its anaylsis online, and an attorney for Judicial Watch has declined to give the name of a third party demographer who created its analysis.
### This news organization used the Python programming language to analyze 5-year census population estimates and a federal voter registration survey.
### The following is a step-by-step review of our analysis:

# Reading in the data:
### First, we used the Python's Pandas library to read in an analyze the data. Its a popular tool used for straight forward and complex data analysis.

In [50]:
import pandas as pd
import numpy as np

### Second, we imported our download of the U.S. Election Assistance Commission's 2018 Election Administration & Voting survey. The commission issues the survey every two years by asking over 6,000 voting jurisidictions to provide various figures regarding their respective electorates.

In [51]:
a1=pd.read_csv("EAVS_2018_for_Public_Release_nolabel v1.1.csv")

In [52]:
a1.head()

,FIPSCode,Jurisdiction_Name,State_Full,State_Abbr,A1a,A1b,A1c,A1Comments,A2a,A2Comments,...,F11d_4,F11d_5,F5_F11Comments,F12a,F12b,F12c,F12d,F12e,F12Comments,F13
0,100100000,AUTAUGA COUNTY,ALABAMA,AL,39770,36391,3379.0,NaN,-88,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,0.0,0.0,NaN,3.0,3.0,1.0,1.0,1.0,NaN,NaN
1,100300000,BALDWIN COUNTY,ALABAMA,AL,155127,141301,13826.0,NaN,-88,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,0.0,0.0,NaN,3.0,3.0,1.0,1.0,1.0,NaN,NaN
2,100500000,BARBOUR COUNTY,ALABAMA,AL,17323,15871,1452.0,NaN,-88,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,0.0,0.0,NaN,3.0,3.0,1.0,1.0,1.0,NaN,NaN
3,100700000,BIBB COUNTY,ALABAMA,AL,14244,13245,999.0,NaN,-88,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,NaN,NaN,NaN,3.0,3.0,1.0,1.0,1.0,NaN,NaN
4,100900000,BLOUNT COUNTY,ALABAMA,AL,39687,37352,2335.0,NaN,-88,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,...,0.0,0.0,NaN,3.0,3.0,1.0,1.0,1.0,NaN,NaN


### The downloaded zip file for the survey included a special code book for the column names. There's over 400 columns in the original table. 
### Judicial Watch claimed it compared census estimates to the total number of registered voters at the county level. The letters also included one column on a particular reason some people were removed. 
### We'll get into the details of those columns later, but for now we'll drop the columns we don't need. In this case, we're keeping columns showing total registered voters, active voters, inactive voters and about 10 columns tracking the removed voters.

In [53]:
a2=a1[["FIPSCode","Jurisdiction_Name","State_Abbr","A1a","A1b",
"A1c","A9a","A9b","A9c","A9d","A9e","A9f","A9g","A9h","A9i","A9j"]]

In [54]:
print(a2.shape)
a2.head()

(6460, 16)


,FIPSCode,Jurisdiction_Name,State_Abbr,A1a,A1b,A1c,A9a,A9b,A9c,A9d,A9e,A9f,A9g,A9h,A9i,A9j
0,100100000,AUTAUGA COUNTY,AL,39770,36391,3379.0,2678.0,592.0,774.0,59.0,862.0,0.0,6.0,NaN,385.0,NaN
1,100300000,BALDWIN COUNTY,AL,155127,141301,13826.0,14145.0,3221.0,3712.0,137.0,5770.0,0.0,22.0,296.0,985.0,2.0
2,100500000,BARBOUR COUNTY,AL,17323,15871,1452.0,1409.0,314.0,496.0,67.0,418.0,0.0,0.0,108.0,1.0,5.0
3,100700000,BIBB COUNTY,AL,14244,13245,999.0,886.0,141.0,411.0,59.0,164.0,0.0,0.0,111.0,NaN,NaN
4,100900000,BLOUNT COUNTY,AL,39687,37352,2335.0,2350.0,323.0,1022.0,296.0,249.0,0.0,0.0,458.0,2.0,NaN


### This is a little easier to read now. The columns A1a through A1c are the columns showing total voters, active voters and inactive voters, while columns A9a through A9j show total voters removed since the last survey and the reasons they were removed.
### Voters can be removed from a jurisdiction's voter roll if they move (A9b), die (A9c), or remain inactive for two federal elections then fail to respond to an address verification letter (A9e). We'll combine the other removed voters columns as "other" for now. 


In [55]:
a2.rename(columns={'A1a':"total_registered_voters",
                   'A1b':'active_voters',
                   'A1c':'inactive_voters',
                   'A9a':'total_removed',
                   'A9b':'moved',
                   'A9c':'died',
                   'A9e':'failed_to_respond'
                  },inplace=True)

### We'll also replace any null values in the remaining columns with zero, then add them together into a new column and drop the ones we started with.

In [56]:
a2=a2.replace(np.nan,0)
a2['removed_other']=a2.A9d+a2.A9f+a2.A9g+a2.A9h+a2.A9i+a2.A9j
a2.drop(columns={'A9d','A9f','A9g','A9h','A9i','A9j'},inplace=True)

In [57]:
a2.head()

,FIPSCode,Jurisdiction_Name,State_Abbr,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other
0,100100000,AUTAUGA COUNTY,AL,39770,36391,3379.0,2678.0,592.0,774.0,862.0,450.0
1,100300000,BALDWIN COUNTY,AL,155127,141301,13826.0,14145.0,3221.0,3712.0,5770.0,1442.0
2,100500000,BARBOUR COUNTY,AL,17323,15871,1452.0,1409.0,314.0,496.0,418.0,181.0
3,100700000,BIBB COUNTY,AL,14244,13245,999.0,886.0,141.0,411.0,164.0,170.0
4,100900000,BLOUNT COUNTY,AL,39687,37352,2335.0,2350.0,323.0,1022.0,249.0,756.0


### Next we'll input our census data. This was taken from census table B01001, 2017 5-year estimates by county. We only added up the totals in that table for men and women 18 or older, then saved it as a seperate file.

In [58]:
population=pd.read_csv("countypop.csv")

In [59]:
print(population.shape)
print(a2.shape)

(3142, 6)
(6460, 11)


### There's only about 3,100 counties in the country, but there's 6,460 jurisdicitions in the EAC survey. While many jurisdictions are counties, some are municipalities and a few are entire states or territories. Nearly all of Wisconsin's jurdiscitions are towns or villages, and a number of those cross county lines.
### First, we need to aggregate our totals to the county level for as many jurisdictions as possible

In [60]:
a2.head()

,FIPSCode,Jurisdiction_Name,State_Abbr,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other
0,100100000,AUTAUGA COUNTY,AL,39770,36391,3379.0,2678.0,592.0,774.0,862.0,450.0
1,100300000,BALDWIN COUNTY,AL,155127,141301,13826.0,14145.0,3221.0,3712.0,5770.0,1442.0
2,100500000,BARBOUR COUNTY,AL,17323,15871,1452.0,1409.0,314.0,496.0,418.0,181.0
3,100700000,BIBB COUNTY,AL,14244,13245,999.0,886.0,141.0,411.0,164.0,170.0
4,100900000,BLOUNT COUNTY,AL,39687,37352,2335.0,2350.0,323.0,1022.0,249.0,756.0


In [61]:
population.head()

,Id,Id2,Geography,adults,county,state
0,0500000US01001,1001,"Autauga County, Alabama",41527,Autauga County,Alabama
1,0500000US01003,1003,"Baldwin County, Alabama",158361,Baldwin County,Alabama
2,0500000US01005,1005,"Barbour County, Alabama",20687,Barbour County,Alabama
3,0500000US01007,1007,"Bibb County, Alabama",17852,Bibb County,Alabama
4,0500000US01009,1009,"Blount County, Alabama",44102,Blount County,Alabama


### Every jurisdiction includes a Federal Information Processing Standard, or FIPS, code. FIPS codes are often used as a way to identify areas at multiple levels in federal data sets. The census population figures use FIPS codes as well, but they used a slightly different, alpha-numeric identifier labeled as "Id". 
### FIPS Codes are typically broken down into a two digit state code, a three digit county code and then various other code variations for localized areas.
### In order to make sure the survey data is as broad as possible, we normalized our "FIPSCode" into its first five digits.
### First, a column was added to count the length of each FIPS code in the survey data. A second column marked True any FIPS codes that were nine digits long, and false for all other values. 

In [62]:
a2['FIPSCode_Len']=a2.FIPSCode.astype(str).str.len()
a2['flentest']=a2.FIPSCode_Len==9
a2.head()

,FIPSCode,Jurisdiction_Name,State_Abbr,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other,FIPSCode_Len,flentest
0,100100000,AUTAUGA COUNTY,AL,39770,36391,3379.0,2678.0,592.0,774.0,862.0,450.0,9,True
1,100300000,BALDWIN COUNTY,AL,155127,141301,13826.0,14145.0,3221.0,3712.0,5770.0,1442.0,9,True
2,100500000,BARBOUR COUNTY,AL,17323,15871,1452.0,1409.0,314.0,496.0,418.0,181.0,9,True
3,100700000,BIBB COUNTY,AL,14244,13245,999.0,886.0,141.0,411.0,164.0,170.0,9,True
4,100900000,BLOUNT COUNTY,AL,39687,37352,2335.0,2350.0,323.0,1022.0,249.0,756.0,9,True


### A complete FIPS code should be 10 digits. A nine digit FIPS Code indicated it should have started with a 0. We replaced the True values in our test column with a zero added to the beginning of the FIPS Code, then replaced False values with the rest of the FIPS column

In [63]:
a2['flentest']=a2.flentest.replace(True,"0"+a2.FIPSCode.astype(str))
a2.flentest.replace(False,a2.FIPSCode,inplace=True)

In [64]:
a2.head()

,FIPSCode,Jurisdiction_Name,State_Abbr,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other,FIPSCode_Len,flentest
0,100100000,AUTAUGA COUNTY,AL,39770,36391,3379.0,2678.0,592.0,774.0,862.0,450.0,9,0100100000
1,100300000,BALDWIN COUNTY,AL,155127,141301,13826.0,14145.0,3221.0,3712.0,5770.0,1442.0,9,0100300000
2,100500000,BARBOUR COUNTY,AL,17323,15871,1452.0,1409.0,314.0,496.0,418.0,181.0,9,0100500000
3,100700000,BIBB COUNTY,AL,14244,13245,999.0,886.0,141.0,411.0,164.0,170.0,9,0100700000
4,100900000,BLOUNT COUNTY,AL,39687,37352,2335.0,2350.0,323.0,1022.0,249.0,756.0,9,0100900000


### If the FIPS code was less than nine digits, it was most likely one of about 1,700 jurisidictions that did not include a state and county FIPS code. 
### Next, we'll print the first five digits of every new FIPS code in the test column into a new "Id" column.

In [65]:
a2['Id']=a2.flentest.astype(str).str[:5]
a2.head()

,FIPSCode,Jurisdiction_Name,State_Abbr,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other,FIPSCode_Len,flentest,Id
0,100100000,AUTAUGA COUNTY,AL,39770,36391,3379.0,2678.0,592.0,774.0,862.0,450.0,9,0100100000,01001
1,100300000,BALDWIN COUNTY,AL,155127,141301,13826.0,14145.0,3221.0,3712.0,5770.0,1442.0,9,0100300000,01003
2,100500000,BARBOUR COUNTY,AL,17323,15871,1452.0,1409.0,314.0,496.0,418.0,181.0,9,0100500000,01005
3,100700000,BIBB COUNTY,AL,14244,13245,999.0,886.0,141.0,411.0,164.0,170.0,9,0100700000,01007
4,100900000,BLOUNT COUNTY,AL,39687,37352,2335.0,2350.0,323.0,1022.0,249.0,756.0,9,0100900000,01009


### Then we drop our extra columns that we no longer need.

In [66]:
a2.drop(columns={'FIPSCode','FIPSCode_Len','flentest'},inplace=True)
a2.head()

,Jurisdiction_Name,State_Abbr,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other,Id
0,AUTAUGA COUNTY,AL,39770,36391,3379.0,2678.0,592.0,774.0,862.0,450.0,01001
1,BALDWIN COUNTY,AL,155127,141301,13826.0,14145.0,3221.0,3712.0,5770.0,1442.0,01003
2,BARBOUR COUNTY,AL,17323,15871,1452.0,1409.0,314.0,496.0,418.0,181.0,01005
3,BIBB COUNTY,AL,14244,13245,999.0,886.0,141.0,411.0,164.0,170.0,01007
4,BLOUNT COUNTY,AL,39687,37352,2335.0,2350.0,323.0,1022.0,249.0,756.0,01009


### Our population data also needs a slight change to its FIPS column. Every county starts with a general, nationwide FIPS code that we can get rid of here.
### We'll also drop some extra columns from the population table. All we really want to keep is the Id column to merge our data, the Geography column to have a clear label field and the total number of adults.

In [67]:
population['Id']=population.Id.str.replace("0500000US",'').astype(str).str.strip()

In [68]:
population.drop(columns={'Id2'},inplace=True)

### Finally, we will add up all of the columns in our survey data while grouping the results by the individual 5-digit fips code.

In [69]:
a2.columns

Index(['Jurisdiction_Name', 'State_Abbr', 'total_registered_voters',
       'active_voters', 'inactive_voters', 'total_removed', 'moved', 'died',
       'failed_to_respond', 'removed_other', 'Id'],
      dtype='object')

In [70]:
a3=a2.groupby(['Id','State_Abbr']).sum().reset_index()

In [71]:
print(a3.shape)
print(population.shape)

(4903, 10)
(3142, 5)


### There are still more rows in the survey than in the population, but that will change after merging the two tables. 
### The survey table currently includes every value under the same FIPS Code assigned to the jurisdictions. That includes every row that didn't have a state or county FIPS code to begin with. 
### The next step is merging the two tables on their Id columns, keeping all of the county population values in tact to track what as excluded.

In [72]:
a3['Id']=a3.Id.str.strip()
a3.drop(columns={'State_Abbr'},inplace=True)

In [73]:
merge1=pd.merge(population,a3,on='Id',how="left")

In [74]:
merge1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3200 entries, 0 to 3199
Data columns (total 13 columns):
Id                         3200 non-null object
Geography                  3200 non-null object
adults                     3200 non-null int64
county                     3200 non-null object
state                      3200 non-null object
total_registered_voters    3100 non-null float64
active_voters              3100 non-null float64
inactive_voters            3100 non-null float64
total_removed              3100 non-null float64
moved                      3100 non-null float64
died                       3100 non-null float64
failed_to_respond          3100 non-null float64
removed_other              3100 non-null float64
dtypes: float64(8), int64(1), object(4)
memory usage: 350.0+ KB


### It appears that 3,042 counties out of 3,142 were able to merge. 

In [75]:
merge1[merge1.total_registered_voters.isna()].sample()

,Id,Geography,adults,county,state,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other
87,02195,"Petersburg Borough, Alaska",2537,Petersburg Borough,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
merge1[merge1.total_registered_voters.isna()].sample()

,Id,Geography,adults,county,state,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other
3173,55135,"Waupaca County, Wisconsin",40812,Waupaca County,Wisconsin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
merge1[merge1.total_registered_voters.isna()].sample()

,Id,Geography,adults,county,state,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other
3161,55111,"Sauk County, Wisconsin",48754,Sauk County,Wisconsin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Alaska was not included because the survey data listed the entire state as the jurisdiction, so there was no county level data to compare it to. As mentioned before, Wisconsin had mostly municipal and multi-county jurisdicitions. Those posed too big of a challenge to keep together in the time available.

In [78]:
merge1.head()

,Id,Geography,adults,county,state,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other
0,01001,"Autauga County, Alabama",41527,Autauga County,Alabama,39770.0,36391.0,3379.0,2678.0,592.0,774.0,862.0,450.0
1,01003,"Baldwin County, Alabama",158361,Baldwin County,Alabama,155127.0,141301.0,13826.0,14145.0,3221.0,3712.0,5770.0,1442.0
2,01005,"Barbour County, Alabama",20687,Barbour County,Alabama,17323.0,15871.0,1452.0,1409.0,314.0,496.0,418.0,181.0
3,01007,"Bibb County, Alabama",17852,Bibb County,Alabama,14244.0,13245.0,999.0,886.0,141.0,411.0,164.0,170.0
4,01009,"Blount County, Alabama",44102,Blount County,Alabama,39687.0,37352.0,2335.0,2350.0,323.0,1022.0,249.0,756.0


### Now we can take a quick look at some of the basic claims included in the Judicial Watch Letters.
### Allegheny was at 98%, Bucks County allegedly had a voter registration rate of 96%, while Chester and Delaware counties were each at 97%, according to Judicial Watch.

In [79]:
merge1['reg_rate']=merge1.total_registered_voters/merge1.adults

In [80]:
pacounties=['Bucks County','Allegheny County','Chester County','Delaware County']

In [81]:
merge1.columns

Index(['Id', 'Geography', 'adults', 'county', 'state',
       'total_registered_voters', 'active_voters', 'inactive_voters',
       'total_removed', 'moved', 'died', 'failed_to_respond', 'removed_other',
       'reg_rate'],
      dtype='object')

In [82]:
merge1[(merge1.state.str.contains('Pennsylvania'))&(merge1.county.isin(pacounties))].groupby('county').reg_rate.sum().round(2)

county
Allegheny County    0.94
Bucks County        0.93
Chester County      0.91
Delaware County     0.92
Name: reg_rate, dtype: float64

### This data has Allegheny at 94%, Bucks at 92%, Chester at 91% and Delaware at 92%.

### The alleged 378 counties with registration rates over 100% are also not represented in this analysis. There are missing counties from this analysis, and 

In [83]:
print(merge1[merge1.reg_rate>=1].sort_values(by='reg_rate',ascending=False).shape)
merge1[merge1.reg_rate>=1].sort_values(by='reg_rate',ascending=False).head()

(277, 14)


,Id,Geography,adults,county,state,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other,reg_rate
1668,30075,"Powder River County, Montana",1342,Powder River County,Montana,13474.0,13474.0,-88.0,2206.0,778.0,317.0,1095.0,16.0,10.040238
1777,31175,"Valley County, Nebraska",3332,Valley County,Nebraska,21891.0,21891.0,-88.0,3604.0,1071.0,624.0,1754.0,155.0,6.569928
1751,31125,"Nance County, Nebraska",2754,Nance County,Nebraska,9748.0,9748.0,-88.0,1267.0,380.0,213.0,612.0,62.0,3.539579
697,17185,"Wabash County, Illinois",9019,Wabash County,Illinois,28828.0,25203.0,3625.0,3049.0,1750.0,830.0,221.0,-544.0,3.196363
2452,46089,"McPherson County, South Dakota",1905,McPherson County,South Dakota,3877.0,3624.0,253.0,154.0,101.0,49.0,0.0,-172.0,2.035171


### There's only about 275 counties with registration rates over 100% here. That's about 100 fewer counties than the counties in Judicial Watch's claims. 

In [84]:
merge1.reg_rate.describe()

count    3100.000000
mean        0.835558
std         0.290674
min        -1.477612
25%         0.781300
50%         0.860342
75%         0.931219
max        10.040238
Name: reg_rate, dtype: float64

### There are some outliers here that are worth removing. The smallest value in the registration rate column is in the negatives, while the highest is at 1,000%. 

In [85]:
merge1[(merge1.reg_rate<=2)&(merge1.reg_rate>=0)].reg_rate.describe()

count    3030.000000
mean        0.847949
std         0.155521
min         0.000913
25%         0.786462
50%         0.862951
75%         0.932330
max         1.631461
Name: reg_rate, dtype: float64

### The difference between the 25th and 75th percentile (known as the interquantile range) is largely unchanged when removing those outliers. The majority of the registration rates in the data fall within 78% to 93%. 
### The standard deviation from the average is a 15-point window on either side of about 85%.
### In other words, the suggestion that the four counties have "absurdly" high registration rates is misleading at best.
### The rates are higher than many others, but they are also far from the upper fringes in this data.

In [86]:
gotletters=["Bucks County, Pennsylvania",
"Delaware County, Pennsylvania",
"Allegheny County, Pennsylvania",
"Chester County, Pennsylvania",
"Jefferson County, Colorado",
"Imperial County, California",
"Monterey County, California",
"Orange County, California",
"Riverside County, California",
"San Diego County, California",
"San Francisco County, California",
"San Mateo County, California",
"Santa Clara County, California",
"Solano County, California",
"Stanislaus County, California",
"Yolo County, California",
"Guilford County, North Carolina",
"Mecklenburg County, North Carolina",
"Fairfax County, Virginia"]

In [87]:
l1=merge1[merge1.Geography.isin(gotletters)]

In [88]:
l1[['Geography','reg_rate']]

,Geography,reg_rate
198,"Imperial County, California",0.789484
212,"Monterey County, California",0.756366
215,"Orange County, California",0.795162
218,"Riverside County, California",0.814172
222,"San Diego County, California",1.013787
223,"San Francisco County, California",0.923834
226,"San Mateo County, California",0.872635
228,"Santa Clara County, California",0.794529
233,"Solano County, California",0.924409
235,"Stanislaus County, California",0.861821


### Registration rates for these 19 counties run the gamut of our earlier standard deviation total. If the average registration rate by county is about 85%, and most counties fall within 15 points of that number, then it's difficult to consider many of these counties as having "unusually high" voter registration rates. 
### If anything, most have fairly mundane rates.

In [89]:
l2=merge1[~merge1.Geography.isin(gotletters)]

In [90]:
l2[['Geography','adults','total_registered_voters','reg_rate']].sort_values(by='reg_rate',ascending=False).head(20)

,Geography,adults,total_registered_voters,reg_rate
1668,"Powder River County, Montana",1342,13474.0,10.040238
1777,"Valley County, Nebraska",3332,21891.0,6.569928
1751,"Nance County, Nebraska",2754,9748.0,3.539579
697,"Wabash County, Illinois",9019,28828.0,3.196363
2452,"McPherson County, South Dakota",1905,3877.0,2.035171
1843,"Harding County, New Mexico",445,726.0,1.631461
2437,"Hanson County, South Dakota",2317,3731.0,1.610272
2729,"McMullen County, Texas",430,687.0,1.597674
300,"San Juan County, Colorado",475,747.0,1.572632
1642,"Fallon County, Montana",2160,3130.0,1.449074


### This is a list of the 20 highest voter registration rates for counties that were not sent letters by Judicial Watch. 
### To be clear, this should not suggest these rates are a true representation of voter registration rates. Ideally, the survey itself would have included some kind of population estimate to draw the rates from. Judicial Watch has not released its methodology, so it's impossible to know exactly why the group chose census data or if they used some additional calculation to adjust their population figures.
### Judicial Watch said it used census data in its anaylsis, therefore, so are we. 

### Judicial Watch also indicated some of these high registration rates were odd for counties of a certain size. The group's letters didn't appear to expand on what it meant, but we'll assume its population.

In [91]:
((l1[['Geography','adults','total_registered_voters','reg_rate']].adults)/1000).describe()

count      19.000000
mean      827.656158
std       719.649894
min       128.223000
25%       392.016500
50%       494.094000
75%       933.641000
max      2555.203000
Name: adults, dtype: float64

### It looks like the smallest county adult population to recieve a letter was a little over 128,000 people.

In [92]:
merge1[(merge1.adults>128000)].shape

(395, 14)

### There are 395 counties that fit that description. 

In [93]:
print(merge1[(merge1.adults>128000)&(merge1.reg_rate>=.9)].state.value_counts().shape)
merge1[(merge1.adults>128000)&(merge1.reg_rate>=.9)].state.value_counts().sum()

(37,)


162

### When we furth remove counties with registration rates estimated by this news organization at or above 90%, we're left with just 162 counties in 37 states.

In [94]:
filtered1=merge1[(merge1.adults>128000)&(merge1.reg_rate>=.9)]

In [95]:
filtered1[filtered1.Geography.isin(gotletters)].shape

(10, 14)

In [96]:
filtered1.shape

(162, 14)

### Only 10 of 19 counties sent letters by Judicial Watch are in this list. That leaves roughly 150 counties that would have seemingly been set a letter over high registration rates with large populations.

In [98]:
filtered1['letter_test']=filtered1.Geography.isin(gotletters)

/Users/ulleryc/data-projects/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [251]:
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())
print(filtered1.sample(n=20).letter_test.value_counts())

False    19
True      1
Name: letter_test, dtype: int64
False    19
True      1
Name: letter_test, dtype: int64
False    20
Name: letter_test, dtype: int64
False    17
True      3
Name: letter_test, dtype: int64
False    20
Name: letter_test, dtype: int64
False    20
Name: letter_test, dtype: int64
False    19
True      1
Name: letter_test, dtype: int64
False    18
True      2
Name: letter_test, dtype: int64
False    19
True      1
Name: letter_test, dtype: int64
False    19
True      1
Name: letter_test, dtype: int64
False    20
Name: letter_test, dtype: int64
False    19
True      1
Name: letter_test, dtype: int64
False    18
True      2
Name: letter_test, dtype: int64
False    18
True      2
Name: letter_test, dtype: int64
False    16
True      4
Name: letter_test, dtype: int64


### The cell above shows the results of a random sampling of all the counties that met our population and registration rate minimums included earlier.
### Out of 15 random samplings of 162 counties, the most counties included in a single sampling was four, while at least three of those samplings returned 20 counties that did not receive letters from Judicial Watch

# Hills like Red Elephants

### At this point in our analysis, there didn't seem to be an apparent rhyme or reason to why these counties were selected by Judicial Watch. At least, not by any connection in the data.
### Judicial Watch's allegations over voter rolls appeared on multiple Reddit.com pages, and a commenter to one of those posts asked a very relevant question: "Remind what number Trump lost the popular vote by?"
### Clinton took the popular vote by 2.8 million votes, and the comment seemed to allude to claims by President Donald Trump that his popular vote loss was due to illegal voting.
### After reading that comment, we went to [MIT's election data project website](https://electionlab.mit.edu/) for voting results by county.

In [102]:
elections1=pd.read_csv("2016-precinct-president.csv",encoding='iso-8859-1')

/Users/ulleryc/data-projects/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,17,19,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [103]:
elections1.head()

,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,...,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
0,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
1,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,P60012234,"JOHNSON, JOHN FITZGERALD MR.",NaN,NaN,NaN,NaN
2,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,P20003984,"STEIN, JILL",NaN,NaN,NaN,NaN
3,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,P80001571,"TRUMP, DONALD J. / MICHAEL R. PENCE",NaN,NaN,NaN,NaN
4,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
elections1.columns

Index(['year', 'stage', 'special', 'state', 'state_postal', 'state_fips',
       'state_icpsr', 'county_name', 'county_fips', 'county_ansi',
       'county_lat', 'county_long', 'jurisdiction', 'precinct', 'candidate',
       'candidate_normalized', 'office', 'district', 'writein', 'party',
       'mode', 'votes', 'candidate_opensecrets', 'candidate_wikidata',
       'candidate_party', 'candidate_last', 'candidate_first',
       'candidate_middle', 'candidate_full', 'candidate_suffix',
       'candidate_nickname', 'candidate_fec', 'candidate_fec_name',
       'candidate_google', 'candidate_govtrack', 'candidate_icpsr',
       'candidate_maplight'],
      dtype='object')

In [112]:
print(elections1.year.value_counts())
print(elections1.stage.value_counts())
print(elections1.special.value_counts())

2016    1989234
Name: year, dtype: int64
gen    1989234
Name: stage, dtype: int64
False    1989234
Name: special, dtype: int64


### We're going to drop the year, stage and special columns since they all contain just one value. After that, we're dropping any columns that only contain null values in them.

In [119]:
elections1.drop(columns={'year','stage','special'},inplace=True)

In [120]:
elections1.dropna(axis=1,how='all',inplace=True)

In [128]:
elections1.columns

Index(['state', 'state_postal', 'state_fips', 'state_icpsr', 'county_name',
       'county_fips', 'county_ansi', 'county_lat', 'county_long',
       'jurisdiction', 'precinct', 'candidate', 'candidate_normalized',
       'office', 'district', 'writein', 'party', 'mode', 'votes',
       'candidate_fec', 'candidate_fec_name'],
      dtype='object')

In [155]:
clinton1=elections1[elections1.candidate=='Hillary Clinton'].groupby('county_fips').votes.sum().to_frame().reset_index()
clinton1.rename(columns={'votes':'clinton'},inplace=True)
trump1=elections1[elections1.candidate=='Donald Trump'].groupby('county_fips').votes.sum().to_frame().reset_index()
trump1.rename(columns={'votes':'trump'},inplace=True)

In [163]:
both1=pd.merge(clinton1,trump1,on='county_fips')

In [175]:
print(both1.info())
both1['Id']="000"+both1.county_fips.astype(int).astype(str)
both1['Id']=both1.Id.str[-5:]
both1.drop(columns={'county_fips'},inplace=True)
both1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3114 entries, 0 to 3113
Data columns (total 4 columns):
county_fips    3114 non-null float64
clinton        3114 non-null int64
trump          3114 non-null int64
Id             3114 non-null object
dtypes: float64(1), int64(2), object(1)
memory usage: 121.6+ KB
None


,clinton,trump,Id
0,5936,18172,01001
1,18458,72883,01003
2,4871,5454,01005
3,1874,6738,01007
4,2150,22808,01009


### We've formated our election resuls to provide county level results for both Trump and Clinton, and we've formated the county_fips column to fit our county level population and voter data.
### Now we'll make a column for trump_wins by testing whether "trump>clinton".

In [176]:
both1['trump_wins']=both1.trump>both1.clinton

### Next, we'll merge this table with our main voter registration and population tables.

In [181]:
electionresults1=pd.merge(merge1,both1,on='Id',how='left')

In [184]:
electionresults1['got_letter']=electionresults1.Geography.isin(gotletters)

In [186]:
electionresults1[(electionresults1.got_letter==True)&(electionresults1.trump_wins==True)]

,Id,Geography,adults,county,state,total_registered_voters,active_voters,inactive_voters,total_removed,moved,died,failed_to_respond,removed_other,reg_rate,clinton,trump,trump_wins,got_letter


### There are no counties where trump_wins and got_letter are both true. Only counties that went to Clinton in 2016 received a letter from the group.
### Of course, the fact that only Clinton counties got letters isn't necessarily a cause for concern. A more relevant question woudl probably be: how often could you choose only counties where Clinton won in 2016 based on population and registration rate standards?

In [191]:
filtered2=electionresults1[(electionresults1.adults>128000)&(electionresults1.reg_rate>=.9)]

In [193]:
print(filtered2.trump_wins.value_counts())
print(filtered2.got_letter.value_counts())

True     86
False    76
Name: trump_wins, dtype: int64
False    152
True      10
Name: got_letter, dtype: int64


### Using the same population minimum and 90% registration rate base, we've got 86 counties that went to Trump in 2016, 76 that went to Clinton, and the same 10 of 152 counties getting letters.

In [200]:
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))
print(filtered2.sample(n=10).trump_wins.value_counts().sort_index(ascending=False))

True     4
False    6
Name: trump_wins, dtype: int64
True     4
False    6
Name: trump_wins, dtype: int64
True     6
False    4
Name: trump_wins, dtype: int64
True     3
False    7
Name: trump_wins, dtype: int64
True     7
False    3
Name: trump_wins, dtype: int64
True     5
False    5
Name: trump_wins, dtype: int64
True     6
False    4
Name: trump_wins, dtype: int64
True     5
False    5
Name: trump_wins, dtype: int64
True     6
False    4
Name: trump_wins, dtype: int64
True     5
False    5
Name: trump_wins, dtype: int64
True     5
False    5
Name: trump_wins, dtype: int64
True     9
False    1
Name: trump_wins, dtype: int64
True     6
False    4
Name: trump_wins, dtype: int64
True     4
False    6
Name: trump_wins, dtype: int64
True     5
False    5
Name: trump_wins, dtype: int64
True     4
False    6
Name: trump_wins, dtype: int64
True     8
False    2
Name: trump_wins, dtype: int64
True     5
False    5
Name: trump_wins, dtype: int64
True     7
False    3
Name: trump_wins, dtype:

### Since there could only be 10 counties that did recieve letters in this list, we're running our sampling again with only 10 spots. Out of 152 counties, eight samples had more Trump than Clinton counties, six had five of each, and another six had more counties where Clinton won in 2016.
### No sample returned results consisting of only one of the two candidates in that election. 
### We then ran the sampling again, for a total of 80 samples. The majority Trump counties made up about 47% of those results, while the Tied and Clinton majorities both came in at just about 26% each.
### If you were to select 10 counties at random, out of a list of counties with a certain population size and registration rate, it appears very likely that you would at least choose five Trump counties and five Clinton counties.
### Judicial Watch chose 10 counties that met our population and registration rate threshold, and then chose nine more under those numbers, and they all happened to go to Clinton in 2016.